In [ ]:
%matplotlib inline
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import minmax_scale
from sklearn.decomposition import PCA


In [ ]:
#carrega o dataset
dados = pd.read_csv('/home/erasmor/Downloads/2017/pc3.csv',sep=",",encoding = 'utf-8',  header=0,na_values='.',dtype={'Label':'category'})

#remove valores infinitos
dados.replace(-np.Inf, np.nan)

#substitui valores NaN
dados.fillna(dados.mean())


In [ ]:
dados.dtypes

In [ ]:
dados.memory_usage(deep=True)

In [ ]:
# verifica quantas instâncias (linhas) e quantos atributos (colunas) a base de dados contém
print("numero de linhas e colunas: ",dados.shape)

In [ ]:
#visualizar distribições por classes contidas no csv - informar nome da classe alvo
print(dados.groupby('Label').size())

In [ ]:
#X_raw_normalize = MinMaxScaler(X_raw_normalize.reshape(0, 1)).reshape(len(X_raw_normalize))
#X_raw_normalizetd2 = (X_raw_normalize - X_raw_normalize.min(axis=0)) / (X_raw_normalize.max(axis=0) - X_raw_normalize.min(axis=0))
# Obtendo os nomes das colunas do DataFrame como uma lista.
cols = list(dados.columns)
# colunas que nao serao normalizadas
cols.remove('Label')

# Copiando os dados e aplicando a normalizacao por reescala nas colunas do DataFrame que contem
# valores continuos. Por padrao, o metodo minmax_scale reescala com min=0 e max=1.
dados = dados[~dados.isin([np.nan, np.inf, -np.inf]).any(1)]
dados[cols] = dados[cols].apply(minmax_scale)


In [ ]:
#define as colunas de atributos e a coluna da classe (de 0 a 72 são atributos e após a 72 é a classe)
# "X_raw" é features/atributos e "y_raw" é target/classe ==> As duas formas abaixo dão certo.
#array = dataset.values
#X_raw = array[:,0:72]
#y_raw = array[:,72]
X_raw = dados.iloc[:, :-1].values # atributos
y_raw = dados.iloc[:, 21].values # classe de ataques
X_raw = np.nan_to_num(X_raw.astype(np.float32))


In [ ]:
#transformar a variável Y com valores categóricos das classses de ataques em valores:
labelencoder_y = LabelEncoder()
y_raw = labelencoder_y.fit_transform(y_raw)
mapping = dict(zip(labelencoder_y.classes_, range(1, len(labelencoder_y.classes_)+1)))

print(mapping)

In [ ]:
# Instanciando um PCA. O parametro n_components indica a quantidade de dimensoes que a base
# original sera reduzida.
pca = PCA(n_components=10, whiten=True,random_state=42)

In [ ]:
# Aplicando o pca na base de dados. O atributo 'values' retorna um numpy.array
# de duas dimensões (matriz) contendo apenas os valores numericos do DataFrame.
X_raw = pca.fit_transform(X_raw)

In [ ]:
def random_sampling_strategie(classifier, X_pool,n_instances,learner=None):
    n_samples = len(X_pool)
    print(n_samples)
    #query_idx = np.random.choice(range(n_samples),size=BATCH_SIZE)
    query_idx = np.random.randint(low=0, high=n_samples, size=BATCH_SIZE)
    return query_idx, X_pool[query_idx]

def random_sampling_tree(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial):
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    from functools import partial
    
    #define nome de arquivos para salvar
    indica_pool=str(idx_dobra)
    
    # recupera as amostras de treino rotuladas iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    X_train_inicial, X_test_inicial, y_train_inicial, y_test_inicial = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw[idx_data[idx_dobra][TRAIN]])) + t_inicial, random_state=42,stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    #print("tamanho de X_train inicial: ",X_train_inicial.shape," tamanho de y_train inicial: ",y_train_inicial.shape)
    #print(y_train_inicial)  
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (será o pool) de acordo com a dobra em uso
    X_pool, y_pool = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos rotulados para o treinamento inicial
    X_train = X_train_inicial
    y_train = y_train_inicial
   
    #instanciando classificadores de aprendizado ativo
    preset_batch = partial(random_sampling_strategie,n_instances=BATCH_SIZE)
    learner_tree = ActiveLearner(estimator=DecisionTreeClassifier(),X_training=X_train, y_training=y_train,query_strategy=preset_batch)
    arquivo_accuracy_tree = open("random_accuracy_performance_tree_dobra_"+indica_pool+".txt","a")
    arquivo_accuracy_history_tree = ("random_accuracy_history_tree_dobra_"+indica_pool+".csv")
    arquivo_f1_score_history_tree = ("random_F1_Score_history_tree_dobra_"+indica_pool+".csv")
    arquivo_tempo_history_tree = ("random_tempo_history_tree_dobra_"+indica_pool+".csv")
    
    #Registro da pontuação na porção de teste com o treinamento inicial
    performance_history_tree=[]
    f1_score_history_tree=[]
    tempo_history_tree=[]
    random_sample_score_tree = learner_tree.score(X_teste, y_teste)
    performance_history_tree.append(random_sample_score_tree)
    predictions = learner_tree.predict(X_teste)
    f1score_tree = 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
    f1_score_history_tree.append(f1score_tree)
    
    #inicio aprendizado ativo
     
    for index in range(N_QUERIES):
        #inicia calculo do tempo de processamento da estratégia (consulta e inserção)
        t1_tree = time.time()
        #recupera amostras do pool baseado na estratégia de consulta
        query_index, query_instance = learner_tree.query(X_pool)
        
        # Ensina ao modelo ActiveLearner o registro solicitado (amostras vão para o topo).
        learner_tree.teach(X=X_pool[query_index].reshape(BATCH_SIZE, -1),y=y_pool[query_index].reshape(BATCH_SIZE, ))
        
        # apaga os registros consultados evitando que seja reconsultados
        X_pool = np.delete(X_pool, query_index, axis=0)
        y_pool = np.delete(y_pool, query_index)
        
        #termina calculo de processamentoda estratégia
        t2_tree = time.time()
        time_elapsed_tree = (t2_tree-t1_tree)
        hours_tree, rem_tree = divmod(time_elapsed_tree, 3600)
        minutes_tree, seconds_tree = divmod(rem_tree, 60)
        tempo_history_tree.append("{:0>2}:{:0>2}:{:0>2}".format(int(hours_tree),int(minutes_tree),int(seconds_tree)))
        # verifica a performance no conjunto de validação visto que o modelo foi treinado com novos dados
        random_sample_score_tree = learner_tree.score(X_teste, y_teste)
        predictions = learner_tree.predict(X_teste)
        clear_output(wait=True)
        print ('Accuracy Random TREE after query no. %d: %f' % (index+1, random_sample_score_tree))
        #arquivo__accuracy_tree.write('Accuracy after query no. %d: %f \n' % (index+1,random_sample_score))
        performance_history_tree.append(random_sample_score_tree)
        #print ('Accuracy after query no. %d: %f' % (index+1, accuracy_score(y_teste, predictions)))
        arquivo_accuracy_tree.write('Accuracy after query no. %d: %f \n' % (index+1,accuracy_score(y_teste, predictions)))
        #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
        arquivo_accuracy_tree.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
        #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
        arquivo_accuracy_tree.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
        #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        #arquivo_accuracy_tree.write('F1 Score after query no. %d: %f \n' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        f1score_tree= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
        f1_score_history_tree.append(f1score_tree)
        arquivo_accuracy_tree.write('F1 score after query no. %d: %f \n' % (index+1, f1score_tree))
        #print ("========================================")
        arquivo_accuracy_tree.write('======================================== \n')
                       
    
    arquivo_accuracy_tree.write("\n Classification report - Random TREE \n")
    arquivo_accuracy_tree.write("\n Avaliação por classe \n")
    arquivo_accuracy_tree.write(classification_report(y_teste, predictions,zero_division=1))  
    np.savetxt(arquivo_accuracy_history_tree, performance_history_tree,delimiter=",")
    np.savetxt(arquivo_f1_score_history_tree, f1_score_history_tree)
    np.savetxt(arquivo_tempo_history_tree,tempo_history_tree,fmt="%s")
    arquivo_accuracy_tree.close()
    


def uncertain_sampling_tree(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST,t_inicial):
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import LinearSVC
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    from modAL.batch import uncertainty_batch_sampling
    from modAL.uncertainty import uncertainty_sampling
    from modAL.uncertainty import classifier_uncertainty
    from modAL.uncertainty import classifier_entropy
    from functools import partial
    
    #define nome de arquivos para salvar
    indica_pool=str(idx_dobra)
    
    # recupera as amostras de treino rotuladas iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    X_train_inicial, X_test_inicial, y_train_inicial, y_test_inicial = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw[idx_data[idx_dobra][TRAIN]])) + t_inicial, random_state=42,stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    #print("tamanho de X_train inicial: ",X_train_inicial.shape," tamanho de y_train inicial: ",y_train_inicial.shape)
    #print(y_train_inicial)  
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (será o pool) de acordo com a dobra em uso
    X_pool, y_pool = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos rotulados para o treinamento inicial
    X_train = X_train_inicial
    y_train = y_train_inicial
   
    #instanciando classificadores de aprendizado ativo
    preset_batch = partial(uncertainty_sampling,n_instances=BATCH_SIZE)
    learner_tree = ActiveLearner(estimator=DecisionTreeClassifier(),X_training=X_train, y_training=y_train,query_strategy=preset_batch)
    arquivo_accuracy_tree = open("uncertain_accuracy_performance_tree_dobra_"+indica_pool+".txt","a")
    arquivo_accuracy_history_tree = ("uncertain_accuracy_history_tree_dobra_"+indica_pool+".csv")
    arquivo_f1_score_history_tree = ("uncertain_F1_Score_history_tree_dobra_"+indica_pool+".csv")
    arquivo_tempo_history_tree = ("uncertain_tempo_history_tree_dobra_"+indica_pool+".csv")
    
    #Registro da pontuação na porção de teste com o treinamento inicial
    performance_history_tree=[]
    f1_score_history_tree=[]
    tempo_history_tree=[]
    uncertain_sample_score_tree = learner_tree.score(X_teste, y_teste)
    performance_history_tree.append(uncertain_sample_score_tree)
    predictions = learner_tree.predict(X_teste)
    f1score_tree = 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
    f1_score_history_tree.append(f1score_tree)
    
    #inicio aprendizado ativo
     
    for index in range(N_QUERIES):
        #inicia calculo do tempo de processamento da estratégia (consulta e inserção)
        t1_tree = time.time()
        #recupera amostras do pool baseado na estratégia de consulta
        query_index, query_instance = learner_tree.query(X_pool)
        
        # Ensina ao modelo ActiveLearner o registro solicitado (amostras vão para o topo).
        learner_tree.teach(X=X_pool[query_index].reshape(BATCH_SIZE, -1),y=y_pool[query_index].reshape(BATCH_SIZE, ))
        
        # apaga os registros consultados evitando que seja reconsultados
        X_pool = np.delete(X_pool, query_index, axis=0)
        y_pool = np.delete(y_pool, query_index)
        
        #termina calculo de processamentoda estratégia
        t2_tree = time.time()
        time_elapsed_tree = (t2_tree-t1_tree)
        hours_tree, rem_tree = divmod(time_elapsed_tree, 3600)
        minutes_tree, seconds_tree = divmod(rem_tree, 60)
        tempo_history_tree.append("{:0>2}:{:0>2}:{:0>2}".format(int(hours_tree),int(minutes_tree),int(seconds_tree)))
        #print("{:0>2}:{:0>2}:{:0>2}".format(int(hours),int(minutes),int(seconds)))        
        # verifica a performance no conjunto de validação visto que o modelo foi treinado com novos dados
        uncertain_sample_score_tree = learner_tree.score(X_teste, y_teste)
        predictions = learner_tree.predict(X_teste)
        clear_output(wait=True)
        print ('Accuracy Uncertain TREE after query no. %d: %f' % (index+1, uncertain_sample_score_tree))
        #arquivo_performance.write('Accuracy after query no. %d: %f \n' % (index+1,uncertain_sample_score))
        performance_history_tree.append(uncertain_sample_score_tree)
        #print ('Accuracy after query no. %d: %f' % (index+1, accuracy_score(y_teste, predictions)))
        arquivo_accuracy_tree.write('Accuracy after query no. %d: %f \n' % (index+1,accuracy_score(y_teste, predictions)))
        #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
        arquivo_accuracy_tree.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
        #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
        arquivo_accuracy_tree.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
        #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        #arquivo_performance.write('F1 Score after query no. %d: %f \n' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        f1score_tree= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
        f1_score_history_tree.append(f1score_tree)
        arquivo_accuracy_tree.write('F1 score after query no. %d: %f \n' % (index+1, f1score_tree))
        #print ("========================================")
        arquivo_accuracy_tree.write('======================================== \n')
                       
    
    arquivo_accuracy_tree.write("\n Classification report - Uncertain TREE \n")
    arquivo_accuracy_tree.write("\n Avaliação por classe \n")
    arquivo_accuracy_tree.write(classification_report(y_teste, predictions,zero_division=1))  
    np.savetxt(arquivo_accuracy_history_tree, performance_history_tree,delimiter=",")
    np.savetxt(arquivo_f1_score_history_tree, f1_score_history_tree)
    np.savetxt(arquivo_tempo_history_tree,tempo_history_tree,fmt="%s")
    arquivo_accuracy_tree.close()

def committee_sampling_tree(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial):
    
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    from modAL.uncertainty import classifier_uncertainty
    from modAL.models import ActiveLearner, Committee
    from modAL.uncertainty import uncertainty_sampling
    from modAL.disagreement import vote_entropy_sampling
    from functools import partial
    
    
        
    #define nome de arquivos para salvar
    indica_pool=str(idx_dobra)
    
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    X_train_inicial, X_test_inicial, y_train_inicial, y_test_inicial = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw[idx_data[idx_dobra][TRAIN]])) + t_inicial, random_state=42, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    #print("tamanho de X_train inicial: ",X_train_inicial.shape," tamanho de y_train inicial: ",y_train_inicial.shape)
    #print(y_train_inicial)  
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (será o pool) de acordo com a dobra em uso
    X_pool, y_pool = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos rotulados para o treinamento inicial
    X_train = X_train_inicial
    y_train = y_train_inicial
    
    # initializing Committee members
    n_members = 3
    learner_list = list()
    
    #instancia classificadores de aprendizado ativo
    preset_batch = partial(vote_entropy_sampling,n_instances=BATCH_SIZE)
    learner_tree = ActiveLearner(estimator=DecisionTreeClassifier(),X_training=X_train, y_training=y_train)
    learner_list.append(learner_tree)
    arquivo_accuracy_tree = open("committee_accuracy_performance_tree_dobra_"+indica_pool+".txt","a")
    arquivo_accuracy_history_tree = ("committee_accuracy_history_tree_dobra_"+indica_pool+".csv")
    arquivo_f1_score_history_tree = ("committee_F1_Score_history_tree_dobra_"+indica_pool+".csv")
    arquivo_tempo_history_tree = ("committee_tempo_history_tree_dobra_"+indica_pool+".csv")
              
    # assembling the committee
    committee = Committee(learner_list=learner_list,query_strategy=preset_batch)
    
    #Registro da pontuação na porção de teste com o treinamento inicial
    performance_history_tree=[]
    f1_score_history_tree=[]
    tempo_history_tree=[]
    uncertain_sample_score_tree = committee.score(X_teste, y_teste)
    performance_history_tree.append(uncertain_sample_score_tree)
    predictions = committee.predict(X_teste)
    f1score_tree = 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
    f1_score_history_tree.append(f1score_tree)
    
        
    for index in range(N_QUERIES):
        #inicia calculo do tempo de processamento da estratégia (consulta e inserção)
        t1_tree = time.time()
        #recupera amostras do pool baseado na estratégia de consulta
        query_index, query_instance = committee.query(X_pool)
            
        # Ensina ao modelo ActiveLearner o registro solicitado (amostras vão para o topo).
        committee.teach(X=X_pool[query_index].reshape(BATCH_SIZE, -1),y=y_pool[query_index].reshape(BATCH_SIZE, ))
       
        # apaga os modelos consultados
        X_pool = np.delete(X_pool, query_index, axis=0)
        y_pool = np.delete(y_pool, query_index)
        
        #termina calculo de processamentoda estratégia
        t2_tree = time.time()
        time_elapsed_tree = (t2_tree-t1_tree)
        hours_tree, rem_tree = divmod(time_elapsed_tree, 3600)
        minutes_tree, seconds_tree = divmod(rem_tree, 60)
        tempo_history_tree.append("{:0>2}:{:0>2}:{:0>2}".format(int(hours_tree),int(minutes_tree),int(seconds_tree)))
        
        # verifica a performance após a inclusão de dados novos 
        committee_sample_score_tree = committee.score(X_teste, y_teste)
        predictions = committee.predict(X_teste)
        clear_output(wait=True)
        print('Accuracy Committee TREE after query no. %d: %f' % (index+1, committee_sample_score_tree))
        arquivo_accuracy_tree.write('Accuracy after query no. %d: %f \n' % (index+1,committee_sample_score_tree))
        performance_history_tree.append(committee_sample_score_tree)
        #print ('Accuracy after query no. %d: %f' % (index+1, accuracy_score(y_test, predictions)))
        #arquivo_performance_tree.write('Accuracy after query no. %d: %f \n' % (index+1,accuracy_score(y_teste, predictions)))
        #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
        arquivo_accuracy_tree.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
        #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
        arquivo_accuracy_tree.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
        #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        f1score_tree= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
        arquivo_accuracy_tree.write('F1 score after query no. %d: %f \n' % (index+1, f1score_tree))
        f1_score_history_tree.append(f1score_tree)
        #print ("========================================")
        arquivo_accuracy_tree.write('======================================== \n')
                
    arquivo_accuracy_tree.write("\n Classification report - Committee TREE \n")
    arquivo_accuracy_tree.write("\n Avaliação por classe \n")
    arquivo_accuracy_tree.write(classification_report(y_teste, predictions,zero_division=1))
    np.savetxt(arquivo_accuracy_history_tree, performance_history_tree,delimiter=",")
    np.savetxt(arquivo_f1_score_history_tree, f1_score_history_tree)
    np.savetxt(arquivo_tempo_history_tree,tempo_history_tree,fmt="%s")
    arquivo_accuracy_tree.close()

def expected_error_reduction_function_tree(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial):
    
    from sklearn.ensemble import RandomForestClassifier
    from modAL.expected_error import expected_error_reduction
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import LinearSVC
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    from copy import deepcopy
    from functools import partial
    
    #indica index da dobra para uso nos arquivos
    indica_pool=str(idx_dobra)
    
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    X_train_inicial, X_test_inicial, y_train_inicial, y_test_inicial = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw[idx_data[idx_dobra][TRAIN]])) + t_inicial,random_state=42, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (será o pool) de acordo com a dobra em uso
    X_pool, y_pool = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos rotulados para o treinamento inicial
    X_train = X_train_inicial
    y_train = y_train_inicial
    
    #instanciando classificadores de aprendizado ativo
    preset_batch = partial(random_sampling_strategie,n_instances=BATCH_SIZE)
    learner_tree = ActiveLearner(estimator=DecisionTreeClassifier(),X_training=X_train, y_training=y_train,query_strategy=preset_batch)
    arquivo_accuracy_tree = open("expected_error_accuracy_performance_tree_dobra_"+indica_pool+".txt","a")
    arquivo_accuracy_history_tree = ("expected_error_accuracy_history_tree_dobra_"+indica_pool+".csv")
    arquivo_f1_score_history_tree = ("expected_error_F1_Score_history_tree_dobra_"+indica_pool+".csv")
    arquivo_tempo_history_tree = ("expected_error_tempo_history_tree_dobra_"+indica_pool+".csv")
        
    #verifica a performance inicial
       
    consultas_validas_tree=0
    performance_history_tree=[]
    f1_score_history_tree=[]
    tempo_history_tree=[]
    unqueried_score_tree = learner_tree.score(X_teste, y_teste)
    predictions = learner_tree.predict(X_teste)
    performance_history_tree.append(unqueried_score_tree)
    f1score_tree = 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
    f1_score_history_tree.append(f1score_tree)
    
    #inicio aprendizado ativo
 
    for index in range(N_QUERIES):
        
        #inicia calculo do tempo de processamento da estratégia (consulta e inserção)
        t1_tree = time.time()
        query_index, query_instance = learner_tree.query(X_pool)
        if expected_error_reduction(learner_tree, X=X_pool[query_index],loss='log',p_subsample=1,random_tie_break=False) >= 0:
            #Ensina ao modelo ActiveLearner as amostras recuperadas (amostras vão para o topo).
            #learner_tree.teach(X=X_pool[query_index],y=y_pool[query_index])
            learner_tree.teach(X=X_pool[query_index].reshape(BATCH_SIZE, -1),y=y_pool[query_index].reshape(BATCH_SIZE, ))
            # grava saida
            new_score_tree = learner_tree.score(X_teste, y_teste)
            performance_history_tree.append(new_score_tree)
            clear_output(wait=True)
            print ('Accuracy Expected Error Reduction TREE after query no. %d: %f' % (index+1, new_score_tree))
            arquivo_accuracy_tree.write('Accuracy after query no. %d: %f \n' % (index+1,new_score_tree))
            predictions = learner_tree.predict(X_teste)
            #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
            arquivo_accuracy_tree.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
            #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
            arquivo_accuracy_tree.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
            #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
            #arquivo_accuracy_tree.write('F1 score after query no. %d: %f \n' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
            f1score_tree= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
            arquivo_accuracy_tree.write('F1 score after query no. %d: %f \n' % (index+1, f1score_tree))
            f1_score_history_tree.append(f1score_tree)
            #print ("========================================")
            arquivo_accuracy_tree.write('======================================== \n')
            #conta quantas consultas foram validas
            consultas_validas_tree=consultas_validas_tree+1
            #apaga registros informativos consultados 
            X_pool = np.delete(X_pool, query_index, axis=0)
            y_pool = np.delete(y_pool, query_index)
            #termina calculo de processamentoda estratégia
            t2_tree = time.time()
            time_elapsed_tree = (t2_tree-t1_tree)
            hours_tree, rem_tree = divmod(time_elapsed_tree, 3600)
            minutes_tree, seconds_tree = divmod(rem_tree, 60)
            tempo_history_tree.append("{:0>2}:{:0>2}:{:0>2}".format(int(hours_tree),int(minutes_tree),int(seconds_tree)))
        
        else:
            #registra score atual para index não ficar menor (sem incremento das amostras recuperadas)
            repete_score_tree = learner_tree.score(X_teste, y_teste)
            performance_history_tree.append(repete_score_tree)
            clear_output(wait=True)
            print ('Accuracy Expected Error Reduction TREE after query no. %d: %f' % (index+1, repete_score_tree))
            arquivo_accuracy_tree.write('Accuracy after query no. %d: %f \n' % (index+1,repete_score_tree))
            predictions = learner_tree.predict(X_teste)
            #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
            arquivo_accuracy_tree.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
            #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
            arquivo_accuracy_tree.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
            #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
            #arquivo_accuracy_tree.write('F1 score after query no. %d: %f \n' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
            f1score_tree= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
            arquivo_accuracy_tree.write('F1 score after query no. %d: %f \n' % (index+1, f1score_tree))
            f1_score_history_tree.append(f1score_tree)
            #print ("========================================")
            arquivo_accuracy_tree.write('======================================== \n')
            #apaga registros informativos consultados 
            X_pool = np.delete(X_pool, query_index, axis=0)
            y_pool = np.delete(y_pool, query_index)
            #termina calculo de processamentoda estratégia
            t2_tree = time.time()
            time_elapsed_tree = (t2_tree-t1_tree)
            hours_tree, rem_tree = divmod(time_elapsed_tree, 3600)
            minutes_tree, seconds_tree = divmod(rem_tree, 60)
            tempo_history_tree.append("{:0>2}:{:0>2}:{:0>2}".format(int(hours_tree),int(minutes_tree),int(seconds_tree)))
        
    arquivo_accuracy_tree.write("\n Classification report - Expected Error Reduction TREE \n")                 
    arquivo_accuracy_tree.write("\n Avaliação por classe \n")
    arquivo_accuracy_tree.write(classification_report(y_teste, predictions,zero_division=1))  
    np.savetxt(arquivo_accuracy_history_tree, performance_history_tree,delimiter=",")
    np.savetxt(arquivo_f1_score_history_tree, f1_score_history_tree)
    np.savetxt(arquivo_tempo_history_tree,tempo_history_tree,fmt="%s")
    arquivo_accuracy_tree.close()

def Expected_Model_Change_tree(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial):
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import LinearSVC
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    from copy import deepcopy
    from functools import partial
    
    #indica index da dobra para uso nos arquivos
    indica_pool=str(idx_dobra)
    
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    X_train_inicial, X_test_inicial, y_train_inicial, y_test_inicial = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw[idx_data[idx_dobra][TRAIN]])) + t_inicial,random_state=42, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    #print("tamanho de X_train inicial: ",X_train_inicial.shape," tamanho de y_train inicial: ",y_train_inicial.shape)
    #print(y_train_inicial)  
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (será o pool) de acordo com a dobra em uso
    X_pool, y_pool = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos rotulados para o treinamento inicial
    X_train = X_train_inicial
    y_train = y_train_inicial
    
    #instanciando classificadores de aprendizado ativo
    preset_batch = partial(random_sampling_strategie,n_instances=BATCH_SIZE)
    learner_tree = ActiveLearner(estimator=RandomForestClassifier(n_estimators=100,random_state=42),X_training=X_train, y_training=y_train,query_strategy=preset_batch)
    arquivo_accuracy_tree = open("model_accuracy_performance_tree_dobra_"+indica_pool+".txt","a")
    arquivo_accuracy_history_tree = ("model_accuracy_history_tree_dobra_"+indica_pool+".csv")
    arquivo_f1_score_history_tree = ("model_F1_Score_history_tree_dobra_"+indica_pool+".csv")
    arquivo_tempo_history_tree = ("model_tempo_history_tree_dobra_"+indica_pool+".csv")
        
    #verifica a performance inicial
    consultas_validas_tree=0
    performance_history_tree=[]
    f1_score_history_tree=[]
    tempo_history_tree=[]
    unqueried_score_tree = learner_tree.score(X_teste, y_teste)
    predictions = learner_tree.predict(X_teste)
    performance_history_tree.append(unqueried_score_tree)
    f1score_tree = 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
    f1_score_history_tree.append(f1score_tree)
    
    #inicio aprendizado ativo
 
    for index in range(N_QUERIES):
        #inicia calculo do tempo de processamento da estratégia (consulta e inserção)
        t1_tree = time.time()
        #n_labeled_examples_news = X_pool.shape[0]
        #training_indices_news = np.random.randint(low=0, high=n_labeled_examples_news, size=BATCH_SIZE)
        #amostra_recuperada_X = X_pool[training_indices_news]
        #amostra_recuperada_y = y_pool[training_indices_news]
        temp_tree = deepcopy(learner_tree)
        #temp_tree.teach(amostra_recuperada_X, amostra_recuperada_y)
        #score_temp_tree = temp_tree.score(X_teste, y_teste)
        #score_learner_tree = learner_tree.score(X_teste, y_teste)
        #recupera amostras do pool baseado na estratégia de consulta
        query_index, query_instance = temp_tree.query(X_pool)
        temp_tree.teach(X=X_pool[query_index].reshape(BATCH_SIZE, -1),y=y_pool[query_index].reshape(BATCH_SIZE, ))
        score_temp_tree = temp_tree.score(X_teste, y_teste)
        score_learner_tree = learner_tree.score(X_teste, y_teste)
        if score_temp_tree > score_learner_tree:
            learner_tree.teach(X=X_pool[query_index].reshape(BATCH_SIZE, -1),y=y_pool[query_index].reshape(BATCH_SIZE, ))
            clear_output(wait=True)
            predictions = learner_tree.predict(X_teste)
            performance_history_tree.append(accuracy_score(y_teste, predictions))
            print ('Accuracy Expected Model TREE after query no. %d: %f' % (index+1, accuracy_score(y_teste, predictions)))
            arquivo_accuracy_tree.write('Accuracy after query no. %d: %f \n' % (index+1,accuracy_score(y_teste, predictions)))
            #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_teste, predictions,average='macro',zero_division=1)))
            arquivo_accuracy_tree.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
            #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
            arquivo_accuracy_tree.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
            #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_teste, predictions,average='macro',zero_division=1)))
            #arquivo_accuracy_tree.write('F1 score after query no. %d: %f \n' % (index+1, f1_score(y_teste, predictions,average='macro',zero_division=1)))
            f1score_tree= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
            f1_score_history_tree.append(f1score_tree)
            arquivo_accuracy_tree.write('F1 score after query no. %d: %f \n' % (index+1, f1score_tree))
            #print ("========================================")
            arquivo_accuracy_tree.write('======================================== \n')
            #apaga registros consultados
            X_pool = np.delete(X_pool, query_index, axis=0)
            y_pool = np.delete(y_pool, query_index)
            #conta quantas consultas foram validas
            consultas_validas_tree=consultas_validas_tree+1
            #termina calculo de processamentoda estratégia
            t2_tree = time.time()
            time_elapsed_tree = (t2_tree-t1_tree)
            hours_tree, rem_tree = divmod(time_elapsed_tree, 3600)
            minutes_tree, seconds_tree = divmod(rem_tree, 60)
            tempo_history_tree.append("{:0>2}:{:0>2}:{:0>2}".format(int(hours_tree),int(minutes_tree),int(seconds_tree)))
        
            
        else:
            clear_output(wait=True)
            predictions = learner_tree.predict(X_teste)
            performance_history_tree.append(accuracy_score(y_teste, predictions))
            print ('Accuracy Expected Model TREE after query no. %d: %f' % (index+1, accuracy_score(y_teste, predictions)))
            arquivo_accuracy_tree.write('Accuracy after query no. %d: %f \n' % (index+1,accuracy_score(y_teste, predictions)))
            #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_teste, predictions,average='macro',zero_division=1)))
            arquivo_accuracy_tree.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
            #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
            arquivo_accuracy_tree.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
            #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_teste, predictions,average='macro',zero_division=1)))
            #arquivo_accuracy_tree.write('F1 score after query no. %d: %f \n' % (index+1, f1_score(y_teste, predictions,average='macro',zero_division=1)))
            f1score_tree= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
            f1_score_history_tree.append(f1score_tree)
            arquivo_accuracy_tree.write('F1 score after query no. %d: %f \n' % (index+1, f1score_tree))
            #print ("========================================")
            arquivo_accuracy_tree.write('======================================== \n')
            
            #apaga registros consultados    
            X_pool = np.delete(X_pool, query_index, axis=0)
            y_pool = np.delete(y_pool, query_index)
            #termina calculo de processamentoda estratégia
            t2_tree = time.time()
            time_elapsed_tree = (t2_tree-t1_tree)
            hours_tree, rem_tree = divmod(time_elapsed_tree, 3600)
            minutes_tree, seconds_tree = divmod(rem_tree, 60)
            tempo_history_tree.append("{:0>2}:{:0>2}:{:0>2}".format(int(hours_tree),int(minutes_tree),int(seconds_tree)))
        
    arquivo_accuracy_tree.write("\n Classification report - Expected Model Change TREE \n")                            
    arquivo_accuracy_tree.write("\n Avaliação por classe \n")
    arquivo_accuracy_tree.write(classification_report(y_teste, predictions,zero_division=1))
    arquivo_accuracy_tree.write('\n Numero de consultas válidas: %d \n' % consultas_validas_tree)
    np.savetxt(arquivo_accuracy_history_tree, performance_history_tree,delimiter=",")
    np.savetxt(arquivo_f1_score_history_tree, f1_score_history_tree)
    np.savetxt(arquivo_tempo_history_tree,tempo_history_tree,fmt="%s")
    arquivo_accuracy_tree.close()



def ranked_sampling_tree(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST,t_inicial):
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import LinearSVC
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    from modAL.batch import uncertainty_batch_sampling
    from functools import partial
        
    #define nome de arquivos para salvar
    indica_pool=str(idx_dobra)
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    X_train_inicial, X_test_inicial, y_train_inicial, y_test_inicial = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw[idx_data[idx_dobra][TRAIN]])) + t_inicial,random_state=42, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (será o pool) de acordo com a dobra em uso
    X_pool, y_pool = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    #isola exemplos rotulados para o treinamento inicial
    X_train = X_train_inicial
    y_train = y_train_inicial
    #instanciando classificadores de aprendizado ativo
    preset_batch = partial(uncertainty_batch_sampling, n_instances=BATCH_SIZE)
    learner_tree = ActiveLearner(estimator=DecisionTreeClassifier(),X_training=X_train, y_training=y_train,query_strategy=preset_batch)
    arquivo_accuracy_tree = open("ranked_accuracy_performance_tree_dobra_"+indica_pool+".txt","a")
    arquivo_accuracy_history_tree = ("ranked_accuracy_history_tree_dobra_"+indica_pool+".csv")
    arquivo_f1_score_history_tree = ("ranked_F1_Score_history_tree_dobra_"+indica_pool+".csv")
    arquivo_tempo_history_tree = ("ranked_tempo_history_tree_dobra_"+indica_pool+".csv")
    
    #Registro da pontuação na porção de teste com o treinamento inicial
    performance_history_tree=[]
    f1_score_history_tree=[]
    tempo_history_tree=[]
    uncertain_sample_score_tree = learner_tree.score(X_teste, y_teste)
    performance_history_tree.append(uncertain_sample_score_tree)
    predictions = learner_tree.predict(X_teste)
    f1score_tree = 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
    f1_score_history_tree.append(f1score_tree)
    
    #inicio aprendizado ativo
     
    for index in range(N_QUERIES):
        #inicia calculo do tempo de processamento da estratégia (consulta e inserção)
        t1_tree = time.time()
        #recupera amostras do pool baseado na estratégia de consulta
        query_index, query_instance = learner_tree.query(X_pool)
        # Ensina ao modelo ActiveLearner o registro solicitado (amostras vão para o topo).
        #learner_tree.teach(X=X_pool[query_index],y=y_pool[query_index])
        learner_tree.teach(X=X_pool[query_index].reshape(BATCH_SIZE, -1),y=y_pool[query_index].reshape(BATCH_SIZE, ))
        # apaga os modelos consultados
        X_pool = np.delete(X_pool, query_index, axis=0)
        y_pool = np.delete(y_pool, query_index, axis=0)
        
        #termina calculo de processamentoda estratégia
        t2_tree = time.time()
        time_elapsed_tree = (t2_tree-t1_tree)
        hours_tree, rem_tree = divmod(time_elapsed_tree, 3600)
        minutes_tree, seconds_tree = divmod(rem_tree, 60)
        tempo_history_tree.append("{:0>2}:{:0>2}:{:0>2}".format(int(hours_tree),int(minutes_tree),int(seconds_tree)))
        
        # verifica a performance no conjunto de validação visto que o modelo foi treinado com novos dados
        uncertain_sample_score_tree = learner_tree.score(X_teste, y_teste)
        predictions = learner_tree.predict(X_teste)
        clear_output(wait=True)
        print ('Accuracy Ranked TREE after query no. %d: %f' % (index+1, uncertain_sample_score_tree))
        arquivo_accuracy_tree.write('Accuracy after query no. %d: %f \n' % (index+1,uncertain_sample_score_tree))
        performance_history_tree.append(uncertain_sample_score_tree)
        #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
        arquivo_accuracy_tree.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
        #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
        arquivo_accuracy_tree.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
        #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        #arquivo_accuracy_tree.write('F1 score after query no. %d: %f \n' % (index+1, f1_score(y_teste, predictions,average='macro',zero_division=1)))
        f1score_tree= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
        arquivo_accuracy_tree.write('F1 score after query no. %d: %f \n' % (index+1, f1score_tree))
        f1_score_history_tree.append(f1score_tree)
        #print ("========================================")
        arquivo_accuracy_tree.write('======================================== \n')
                       
    arquivo_accuracy_tree.write("\n Classification report - Ranked TREE \n")    
    arquivo_accuracy_tree.write("\n Avaliação por classe \n")
    arquivo_accuracy_tree.write(classification_report(y_teste, predictions,zero_division=1))   
    np.savetxt(arquivo_accuracy_history_tree, performance_history_tree,delimiter=",")
    np.savetxt(arquivo_f1_score_history_tree, f1_score_history_tree)
    np.savetxt(arquivo_tempo_history_tree,tempo_history_tree,fmt="%s")
    arquivo_accuracy_tree.close()
    
    
import time
import sys
from datetime import datetime
from datetime import date
import threading
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit, train_test_split
from modAL.uncertainty import classifier_uncertainty
from modAL.models import ActiveLearner
#from modAL.uncertainty import uncertainty_sampling
from timeit import Timer
import time
import functools
# importing the multiprocessing module
import multiprocessing

#inicia relogio
t1 = time.time()
# Define o tamanho das divisões feitas no dataset (cross-validation)
n_dobras = 5
# Define Tamanho inicial da amostra (toda estratégia parte de um tamanho mínimo aleatório).
t_inicial = 10

#define array de indices das partições
idx_data =[]
# cross validation bags - n_splits
data_cv = StratifiedShuffleSplit(n_splits= n_dobras, train_size=221, test_size=221, random_state=42)
#data_cv = StratifiedShuffleSplit(n_splits= n_dobras,random_state=42)
data_cv.get_n_splits(X_raw, y_raw)
# chame a instância e gere os dados sobre a base original
type(data_cv.split(X_raw, y_raw))
# dividir os dados - A função split.split () retorna índices para amostras de treino e amostras de teste. 
# Ele examinará o número de validação cruzada especificado e retornará cada vez que treinar 
# e testar os índices de amostra usando os conjuntos de dados de treinamento e teste que podem 
# ser criados filtrando o conjunto de dados inteiro. Por exemplo idx_data[0][1], o primeiro indice faz referencia
# a dobra e o segundo indice faz referencia a posição da dobra (0 = treino e 1 = teste). Logo TRAIN=0 e TEST=1.
for train_index, test_index in data_cv.split(X_raw,y_raw):
    #print("TRAIN:", train_index, "TEST:", test_index)
    #print("n_split",n_splits,"TRAIN:", train_index, "TEST:", test_index)
    idx_data.append([train_index, test_index])
#verifica tamanho das dobras (numero de instâncias de cada dobra)
#print("tamanho de cada dobra de treino: ",idx_data[1][0].shape," tamanho de cada dobra de teste: ",idx_data[1][1].shape)

TRAIN =0
TEST =1

BATCH_SIZE = 4
N_RAW_SAMPLES = 220
N_QUERIES = N_RAW_SAMPLES // BATCH_SIZE

#chama procedimento de aprendizado para todas as dobras

for idx_dobra in range(n_dobras):    
    threading.Thread(target=random_sampling_tree(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial)).start()
    threading.Thread(target=uncertain_sampling_tree(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial)).start()
    threading.Thread(target=committee_sampling_tree(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial)).start()
    threading.Thread(target=expected_error_reduction_function_tree(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial)).start()
    threading.Thread(target=Expected_Model_Change_tree(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial)).start()
    threading.Thread(target=ranked_sampling_tree(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial)).start()
    
    
    
        
# todos os processos finalizados
print("Terminado!")
        
t2 = time.time()
time_elapsed = (t2-t1)
hours, rem = divmod(time_elapsed, 3600)
minutes, seconds = divmod(rem, 60)
print("Tempo total: {:0>2}:{:0>2}:{:0>2}".format(int(hours),int(minutes),int(seconds)))